In [1]:
import pandas as pd
import numpy as np
import configparser 

In [2]:
config = configparser.ConfigParser()
config.read('config.cfg')

['config.cfg']

In [3]:
config.get('RDS', 'DB_HOST')

'db-sakila-a.c7e08ucgux67.us-east-2.rds.amazonaws.com'

In [16]:
mysql_driver = f"""mysql+pymysql://{config.get('RDS', 'DB_USERNAME')}:{config.get('RDS', 'DB_PASSWORD')}@{config.get('RDS', 'DB_HOST')}:{config.get('RDS', 'DB_PORT')}/{config.get('RDS', 'DB_NAME')}"""

### Cargamos Archivo country.csv

In [12]:
country_data = pd.read_csv('./Data/sakila-db/data/country.csv')
country_data.head()

,country_id,country,last_update
0,1,Afghanistan,2006-02-15 04:44:00
1,2,Algeria,2006-02-15 04:44:00
2,3,American Samoa,2006-02-15 04:44:00
3,4,Angola,2006-02-15 04:44:00
4,5,Anguilla,2006-02-15 04:44:00


In [17]:
country_data.to_sql('country', mysql_driver, index=False, if_exists='append')

109

### Cargamos archivo city

In [18]:
city_data = pd.read_csv('./Data/sakila-db/data/city.csv')
city_data.head()

,city_id,city,country_id,last_update
0,1,A Corua (La Corua),87,2006-02-15 04:45:25
1,2,Abha,82,2006-02-15 04:45:25
2,3,Abu Dhabi,101,2006-02-15 04:45:25
3,4,Acua,60,2006-02-15 04:45:25
4,5,Adana,97,2006-02-15 04:45:25


In [19]:
city_data.to_sql('city', mysql_driver, index=False, if_exists='append')

602

### Leemos datos de tabla country

In [20]:
sql_query = 'SELECT * FROM country;'
df_country = pd.read_sql(sql_query, mysql_driver)
df_country.head()

,country_id,country,last_update
0,1,Afghanistan,2006-02-15 04:44:00
1,2,Algeria,2006-02-15 04:44:00
2,3,American Samoa,2006-02-15 04:44:00
3,4,Angola,2006-02-15 04:44:00
4,5,Anguilla,2006-02-15 04:44:00


In [21]:
sql_query = 'SELECT * FROM city;'
df_city = pd.read_sql(sql_query, mysql_driver)
df_city.head()

,city_id,city,country_id,last_update
0,1,A Corua (La Corua),87,2006-02-15 04:45:25
1,2,Abha,82,2006-02-15 04:45:25
2,3,Abu Dhabi,101,2006-02-15 04:45:25
3,4,Acua,60,2006-02-15 04:45:25
4,5,Adana,97,2006-02-15 04:45:25


### Merge country_city

In [24]:
df_country_city = df_city.merge(df_country, left_on='country_id', right_on='country_id', how='inner')
df_country_city = df_country_city[['city', 'country']]
df_country_city.head()

,city,country
0,A Corua (La Corua),Spain
1,Donostia-San Sebastin,Spain
2,Gijn,Spain
3,Ourense (Orense),Spain
4,Santiago de Compostela,Spain


In [27]:
df_country_city['country'].value_counts()[0:5]

country
India            60
China            53
United States    35
Japan            31
Mexico           30
Name: count, dtype: int64